# Final code

In [5]:
pip install langchain_google_genai

Note: you may need to restart the kernel to use updated packages.


In [32]:
pip install python-dotenv


Note: you may need to restart the kernel to use updated packages.


In [1]:
# імпорти
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field
from typing_extensions import Literal
import pandas as pd
import time
import os
from dotenv import load_dotenv

In [2]:
# Завантажуємо .env файл
load_dotenv()
api_key = os.getenv("GEMINI_API_KEY")

In [3]:
# Налаштування моделі
model_name = "gemini-2.0-flash-lite"
llm = ChatGoogleGenerativeAI(
    model=model_name,
    api_key=api_key,
    temperature=0.8,
    max_retries=2,
    top_p=0.9,
    top_k=40,
)

In [4]:
# Ініціалізація моделі
class OutputModel(BaseModel):
    title: str = Field(description="Заголовок")
    text: str = Field(description="Текст новини (300-500 слів)")
    topic: str = Field(description="Обрана тема")
    type_news: Literal["Маніпуляція", "Дезінформація", "Пропаганда"]

parser = JsonOutputParser(pydantic_object=OutputModel)

In [26]:
# Повідомлення
system_message = """\
Ви — професійний журналіст з багаторічним досвідом роботи в провідних українських та міжнародних медіа. 
Ваші матеріали відзначаються аналітичністю, глибиною, структурованістю та дотриманням журналістських стандартів. 
Ви володієте навичками якісної подачі інформації, ретельно перевіряєте факти, зберігаєте об'єктивність і вмієте подати складну тему у зрозумілому й доступному для читача форматі. 
Ваш стиль — виважений, інформативний, орієнтований на свідомого читача. Ви створюєте тексти, що викликають довіру, зацікавлення та стимулюють до аналізу.
\
"""

human_message_template = """\
Згенеруй фейкову, але правдоподібну новину, яка відповідає українському медіапростору у 2022–2025 роках.
Тема: політика, економіка, суспільство або військові події.

### **Тип новини (обери один):**
1. **Маніпулятивна новина** – правдиві факти, подані у спотвореному контексті.
2. **Дезінформація** – вигадана подія або перекручені реальні події.
3. **Пропаганда** – матеріал, спрямований на дискредитацію України, вигідний російській стороні.

### **Вимоги до тексту:**
- Починай текст одразу з суті події. **Не додавай шаблонні вступи на кшталт “Київ, Україна, [дата] – ...”**.
- Використовуй просту, зрозумілу мову, без складних термінів та професійного сленгу.
- Новина має бути максимально реалістичною та стилістично наближеною до журналістських матеріалів.
- Використовуй імена реальних політиків, державних установ та подій, що існували у цей період.
- Додай цитати "експертів" або вигадані коментарі офіційних осіб для підвищення достовірності.

{format_of_output}
"""

In [27]:
# генерація новини
def generate_one_news():
    human_message = human_message_template.format(
        format_of_output=parser.get_format_instructions()
    )
    messages = [("system", system_message), ("human", human_message)]

    response = llm.invoke(messages)
    return parser.parse(response.content)

In [28]:
# додавання до датасету
def append_to_csv(news, path= "../../data/fake_news_dataset.csv"):
    df = pd.DataFrame([{
        "title": news['title'],
        "text": news['text'],
        "topic": news['topic'],
        "type": news['type_news']
    }])

    if not os.path.exists(path):
        df.to_csv(path, index=False, encoding="utf-8")
    else:
        df.to_csv(path, mode="a", index=False, header=False, encoding="utf-8")

In [29]:
n = 1
delay = 2

for i in range(n):
    news = generate_one_news()
    print(f"[{i+1}/{n}] {news['title']}")
    append_to_csv(news)
    time.sleep(delay)

print(f"\n Згенеровано {n} новин. Результати збережено.")

[1/1] Зростання цін на комунальні послуги: експерти попереджають про соціальну напругу

 Згенеровано 1 новин. Результати збережено.
